# US Name
Model estimate Estimate sign of effect


# Description

None

# Metadata

- Key: 242_esg_metadata 
- Epic: Models
- US: Estimate sign of effect
- Task tag: #draft, #polymer, #sign-of-effect
- Analytics reports: 

# Input

## Table/file

**Name**

None

**Github**

- https://github.com/thomaspernet/esg_metadata/blob/master/02_data_analysis/01_model_train_evaluate/01_sign_of_effect/00_sign_of_effect_classification.md



# Connexion server

In [8]:
from awsPy.aws_authorization import aws_connector
from awsPy.aws_s3 import service_s3
from awsPy.aws_glue import service_glue
from pathlib import Path
import pandas as pd
import numpy as np
#import seaborn as sns
import os, shutil, json
import sys

path = os.getcwd()
parent_path = str(Path(path).parent.parent.parent)


name_credential = 'financial_dep_SO2_accessKeys.csv'
region = 'eu-west-2'
bucket = 'datalake-london'
path_cred = "{0}/creds/{1}".format(parent_path, name_credential)

In [9]:
con = aws_connector.aws_instantiate(credential = path_cred,
                                       region = region)
client= con.client_boto()
s3 = service_s3.connect_S3(client = client,
                      bucket = bucket, verbose = False)
glue = service_glue.connect_glue(client = client) 

In [10]:
pandas_setting = True
if pandas_setting:
    #cm = sns.light_palette("green", as_cmap=True)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_colwidth', None)

In [11]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'


# Load tables

Since we load the data as a Pandas DataFrame, we want to pass the `dtypes`. We load the schema from Glue to guess the types

In [12]:
db = 'esg'
table = 'meta_analysis_esg_cfp'

In [13]:
dtypes = {}
schema = (glue.get_table_information(database = db,
                           table = table)
          ['Table']['StorageDescriptor']['Columns']
         )
for key, value in enumerate(schema):
    if value['Type'] in ['varchar(12)',
                         'varchar(3)',
                        'varchar(14)', 'varchar(11)']:
        format_ = 'string'
    elif value['Type'] in ['decimal(21,5)', 'double', 'bigint', 'int', 'float']:
        format_ = 'float'
    else:
        format_ = value['Type'] 
    dtypes.update(
        {value['Name']:format_}
    )

In [14]:
download_data = True
filename = 'df_{}'.format(table)
full_path_filename = 'SQL_OUTPUT_ATHENA/CSV/{}.csv'.format(filename)
path_local = os.path.join(str(Path(path).parent.parent.parent), 
                              "00_data_catalog/temporary_local_data")
df_path = os.path.join(path_local, filename + '.csv')
if download_data:
    
    s3 = service_s3.connect_S3(client = client,
                          bucket = bucket, verbose = False)
    query = """
    WITH test as (
  SELECT 
    *, concat(environmnental,  social, governance) as filters
  FROM {}.{} 
  WHERE 
    first_date_of_observations IS NOT NULL 
    and last_date_of_observations IS NOT NULL 
    and adjusted_model != 'TO_REMOVE' 
) 
SELECT 
  filters, to_remove, test.id, image, row_id_excel, row_id_google_spreadsheet,
       table_refer, incremental_id, paper_name, publication_name,
       rank, sjr, sjr_best_quartile, h_index, total_docs_2020,
       total_docs_3years, total_refs, total_cites_3years,
       citable_docs_3years, cites_doc_2years, country,
       publication_year, publication_type, cnrs_ranking, peer_reviewed,
       study_focused_on_social_environmental_behaviour, type_of_data,
       study_focusing_on_developing_or_developed_countries, regions,
       first_date_of_observations, last_date_of_observations,
       windows, avg_windows, adjusted_model_name,
       adjusted_model, dependent, adjusted_dependent, independent,
       adjusted_independent, 
       CASE WHEN social = 'True' THEN 'YES' ELSE 'NO' END AS social,
       CASE WHEN environmnental = 'True' THEN 'YES' ELSE 'NO' END AS environmnental,
       CASE WHEN governance = 'True' THEN 'YES' ELSE 'NO' END AS governance,
       CASE WHEN financial_crisis = True THEN 'YES' ELSE 'NO' END AS financial_crisis,
       CASE WHEN kyoto = True THEN 'YES' ELSE 'NO' END AS kyoto,
       lag,
       interaction_term, quadratic_term, n, r2, beta,
       sign_of_effect, significant, final_standard_error,
       to_check_final, weight 
FROM 
  test 
  LEFT JOIN (
    SELECT 
      id, 
      COUNT(*) as weight 
    FROM 
      test 
    GROUP BY 
      id
  ) as c on test.id = c.id
  WHERE filters != 'TrueTrueTrue' and filters != 'FalseFalseFalse'

    """.format(db, table)
    try:
        df = (s3.run_query(
            query=query,
            database=db,
            s3_output='SQL_OUTPUT_ATHENA',
            filename=filename,  # Add filename to print dataframe
            destination_key='SQL_OUTPUT_ATHENA/CSV',  #Use it temporarily
            dtype = dtypes
        )
                )
    except:
        pass
    s3.download_file(
        key = full_path_filename
    )
    shutil.move(
        filename + '.csv',
        os.path.join(path_local, filename + '.csv')
    )
    s3.remove_file(full_path_filename)
df.head(2)

,filters,to_remove,id,image,row_id_excel,row_id_google_spreadsheet,table_refer,incremental_id,paper_name,publication_name,rank,sjr,sjr_best_quartile,h_index,total_docs_2020,total_docs_3years,total_refs,total_cites_3years,citable_docs_3years,cites_doc_2years,country,publication_year,publication_type,cnrs_ranking,peer_reviewed,study_focused_on_social_environmental_behaviour,type_of_data,study_focusing_on_developing_or_developed_countries,regions,first_date_of_observations,last_date_of_observations,windows,avg_windows,adjusted_model_name,adjusted_model,dependent,adjusted_dependent,independent,adjusted_independent,social,environmnental,governance,financial_crisis,kyoto,lag,interaction_term,quadratic_term,n,r2,beta,sign_of_effect,significant,final_standard_error,to_check_final,weight
0,FalseTrueFalse,TO_KEEP,76,NaN,https://docs.google.com/spreadsheets/d/11A3l50yfiGxxRuyV-f3WV9Z-4DcsQLYW6XBl4a7U4bQ/edit#gid=18346659&range=B1000,https://docs.google.com/spreadsheets/d/1d66_CVtWni7wmKlIMcpaoanvT2ghmjbXARiHgnLWvUw/edit#gid=899172650&range=B513,table 9,512,Corporate social responsibility and corporate financial performance: Evidence from Korea,australian journal of management,11659.0,0.419,Q2,39.0,49.0,94.0,3718.0,182.0,94.0,2.06,United Kingdom,2010.0,Journal Article,0.0,Yes,Environmental and Social,Panel data,Developed,ASIA AND PACIFIC,2002,2008,6,3,POOLED OLS,POOLED OLS,ROA,ROA,SW,SOCIAL,YES,NO,NO,YES,YES,yes,no,no,1222.0,0.247,0.002,POSITIVE,NO,3.38,NO,12
1,FalseTrueFalse,TO_KEEP,76,NaN,https://docs.google.com/spreadsheets/d/11A3l50yfiGxxRuyV-f3WV9Z-4DcsQLYW6XBl4a7U4bQ/edit#gid=18346659&range=B1000,https://docs.google.com/spreadsheets/d/1d66_CVtWni7wmKlIMcpaoanvT2ghmjbXARiHgnLWvUw/edit#gid=899172650&range=B514,table 9,513,Corporate social responsibility and corporate financial performance: Evidence from Korea,australian journal of management,11659.0,0.419,Q2,39.0,49.0,94.0,3718.0,182.0,94.0,2.06,United Kingdom,2010.0,Journal Article,0.0,Yes,Environmental and Social,Panel data,Developed,ASIA AND PACIFIC,2002,2008,6,3,POOLED OLS,POOLED OLS,ROE,ROE,SW,SOCIAL,YES,NO,NO,YES,YES,yes,no,no,1222.0,0.216,0.001,POSITIVE,NO,2.78,NO,12


In [15]:
#df['adjusted_model'].unique()

In [16]:
#df['financial_crisis'].unique()

In [17]:
pd.DataFrame(schema)

,Name,Type,Comment
0,to_remove,string,to remove
1,id,string,paper ID
2,image,string,Link row data image
3,row_id_excel,string,link to original row
4,row_id_google_spreadsheet,string,Google spreadsheet link to raw data
5,table_refer,string,table number in the paper
6,incremental_id,string,row id
7,paper_name,string,Paper name
8,publication_name,string,publication name
9,rank,bigint,journal rank


## unbalanced ID

In [18]:
#df['weight'].describe()

## Schema Latex table

To rename a variable, please use the following template:

```
{
    'old':'XX',
    'new':'XX_1'
    }
```

if you need to pass a latex format with `\`, you need to duplicate it for instance, `\text` becomes `\\text:

```
{
    'old':'working\_capital\_i',
    'new':'\\text{working capital}_i'
    }
```

Then add it to the key `to_rename`

In [19]:
add_to_dic = False
if add_to_dic:
    if os.path.exists("schema_table.json"):
        os.remove("schema_table.json")
    data = {'to_rename':[], 'to_remove':[]}
    dic_rename = [
        {
        'old':'working\_capital\_i',
        'new':'\\text{working capital}_i'
        },
    ]

    data['to_rename'].extend(dic_rename)
    with open('schema_table.json', 'w') as outfile:
        json.dump(data, outfile)

In [20]:
sys.path.append(os.path.join(parent_path, 'utils'))
import latex.latex_beautify as lb
#%load_ext autoreload
#%autoreload 2

In [21]:
#!conda install -c conda-forge r-lmtest -y

In [22]:
options(warn=-1)
library(tidyverse)
library("sandwich")
library("lmtest")
#library(lfe)
#library(lazyeval)
#library(nnet)
library('progress')
path = "../../../utils/latex/table_golatex.R"
source(path)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1       ✔ purrr   0.3.2  
✔ tibble  2.1.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Please cite as: 

 Hlavac, Marek (2018). stargazer: Well-Formatted Regression and Summary Statistics Tables.
 R package version 5.2.2. https://CRAN.R-project.org/package=stargazer 



In [23]:
%get df_path
df_final <- read_csv(df_path) %>%
mutate_if(is.character, as.factor) %>%
mutate(
    sign_of_effect = relevel(sign_of_effect, ref='INSIGNIFICANT'),
    adjusted_model = relevel(adjusted_model, ref='OTHER'),
    adjusted_dependent = relevel(adjusted_dependent, ref='OTHER'),
      id = as.factor(id),
    governance = relevel(as.factor(governance), ref = 'NO'),
    social = relevel(as.factor(social), ref = 'NO'),
    environmnental =relevel(as.factor(environmnental), ref = 'NO'),
    financial_crisis =relevel(as.factor(financial_crisis), ref = 'NO'),
    kyoto =relevel(as.factor(kyoto), ref = 'NO'),
) 

Parsed with column specification:
cols(
  .default = col_character(),
  id = col_double(),
  incremental_id = col_double(),
  rank = col_double(),
  sjr = col_double(),
  h_index = col_double(),
  total_docs_2020 = col_double(),
  total_docs_3years = col_double(),
  total_refs = col_double(),
  total_cites_3years = col_double(),
  citable_docs_3years = col_double(),
  cites_doc_2years = col_double(),
  publication_year = col_double(),
  cnrs_ranking = col_double(),
  first_date_of_observations = col_double(),
  last_date_of_observations = col_double(),
  windows = col_double(),
  avg_windows = col_double(),
  n = col_double(),
  r2 = col_double(),
  beta = col_double()
  # ... with 2 more columns
)
See spec(...) for full column specifications.


In [24]:
glimpse(df_final)

Observations: 650
Variables: 55
$ filters                                             <fct> FalseTrueFalse, F…
$ to_remove                                           <fct> TO_KEEP, TO_KEEP,…
$ id                                                  <fct> 76, 76, 76, 78, 7…
$ image                                               <fct> NA, NA, NA, https…
$ row_id_excel                                        <fct> https://docs.goog…
$ row_id_google_spreadsheet                           <fct> https://docs.goog…
$ table_refer                                         <fct> table 9, table 9,…
$ incremental_id                                      <dbl> 512, 513, 514, 51…
$ paper_name                                          <fct> Corporate social …
$ publication_name                                    <fct> australian journa…
$ rank                                                <dbl> 11659, 11659, 116…
$ sjr                                                 <dbl> 0.419, 0.419, 0.4…
$ sjr_best_quartile 

In [25]:
transpose(df_final %>% 
    select_if(function(x) any(is.na(x))) %>% 
    summarise_each(funs(sum(is.na(.)))))

[[1]]
[[1]]$image
[1] 139

[[1]]$rank
[1] 96

[[1]]$sjr
[1] 96

[[1]]$sjr_best_quartile
[1] 96

[[1]]$h_index
[1] 96

[[1]]$total_docs_2020
[1] 96

[[1]]$total_docs_3years
[1] 96

[[1]]$total_refs
[1] 96

[[1]]$total_cites_3years
[1] 96

[[1]]$citable_docs_3years
[1] 96

[[1]]$cites_doc_2years
[1] 96

[[1]]$country
[1] 96

[[1]]$publication_type
[1] 8

[[1]]$cnrs_ranking
[1] 6

[[1]]$peer_reviewed
[1] 20

[[1]]$type_of_data
[1] 2

[[1]]$study_focusing_on_developing_or_developed_countries
[1] 10

[[1]]$lag
[1] 3

[[1]]$interaction_term
[1] 3

[[1]]$quadratic_term
[1] 3

[[1]]$n
[1] 6

[[1]]$r2
[1] 99

[[1]]$final_standard_error
[1] 135

GLM does not clustered the standard error so, we compute it by hand

In [26]:
se_robust <- function(x)
  coeftest(x, vcov. = sandwich::sandwich
          )[, 2]
se_robust_clustered <- function(x)
  coeftest(x,
         vcov. = vcovCL(t_2, cluster = df_final %>% filter(adjusted_model != 'TO_REMOVE') %>% select(id), type = "HC0")
        )[, 2]

# Multinomial

**Note**: 
- comparison group "INSIGNIFICANT" 
- Standard error not robust
- Results are relative risk ratios:
    - Relative risk ratios allow an easier interpretation of the logit coefficients. They are the
exponentiated value of the logit coefficients

In [27]:
library(nnet)

In [46]:
(
    df
    ['sign_of_effect']
    .value_counts()
)

INSIGNIFICANT    306
POSITIVE         239
NEGATIVE         105
Name: sign_of_effect, dtype: int64

In [29]:
(
    df
    .groupby(['sign_of_effect'])['governance']
    .value_counts()
    .unstack(-1)
)

governance,NO,YES
sign_of_effect,,
INSIGNIFICANT,286,20
NEGATIVE,103,2
POSITIVE,220,19


In [30]:
(
    df
    .groupby(['adjusted_model','sign_of_effect'])['governance']
    .value_counts()
    .rename('count')
    .reset_index()
    .set_index(['governance', 'sign_of_effect', 'adjusted_model'])
    .unstack(-1)
    .style
    .format("{0:,.0f}")
)

In [31]:
(
    df
    .groupby(['adjusted_model','sign_of_effect'])['social']
    .value_counts()
    .rename('count')
    .reset_index()
    .set_index(['social', 'sign_of_effect', 'adjusted_model'])
    .unstack(-1)
    .style
    .format("{0:,.0f}")
)

In [32]:
(
    df
    .groupby(['adjusted_model','sign_of_effect'])['environmnental']
    .value_counts()
    .rename('count')
    .reset_index()
    .set_index(['environmnental', 'sign_of_effect', 'adjusted_model'])
    .unstack(-1)
    .style
    .format("{0:,.0f}")
)

**How to read**

- Categorical:
    - Keeping all other variables constant, if the analysis uses FIXED EFFECT model, there are 2.71 times more likely to stay in the NEGATIVE sign category as compared to the OTHER model category. The coefficient, however, is not significant. (Col 1)
- Continuous:
    - Keeping all other variables constant, if the SJR score increases one unit, there is 1.003 times more likely to stay in the POSITIVE sign category as compared to the OTHER model category y (the risk or odds is 2% higher).. The coefficient is significant.
    
Here, OTHER means insignificant

Currently, issue with:

- governance 
- full inclusion dummy -> probably collinearity need to check

In [33]:
#
t_1 <- multinom(sign_of_effect ~ adjusted_model+ social  +#adjusted_dependent+
           sjr, data = df_final, trace = FALSE)    
t_1.rrr <- exp(coef(t_1))
#
t_2 <- multinom(sign_of_effect ~ adjusted_model+ environmnental  +#adjusted_dependent+
           sjr, data = df_final, trace = FALSE)    
t_2.rrr <- exp(coef(t_2))
#
t_3 <- multinom(sign_of_effect ~ adjusted_model+ governance  +#adjusted_dependent+
           sjr, data = df_final, trace = FALSE)    
t_3.rrr <- exp(coef(t_3))
#
t_4 <- multinom(sign_of_effect ~ adjusted_model+ social +environmnental+ governance  +#adjusted_dependent+
           sjr, data = df_final, trace = FALSE)    
t_4.rrr <- exp(coef(t_4))

dep <- "Dependent variable: Sign insignificant"

list_final <- list(t_1, t_2, t_3, t_4)
list_final.rrr <-list(t_1.rrr, t_2.rrr, t_3.rrr, t_4.rrr)
stargazer(list_final,
          type = "text", 
          coef=list_final.rrr,
          omit = "id",
          style = "qje")


                             NEGATIVE  POSITIVE  NEGATIVE  POSITIVE  NEGATIVE  POSITIVE    NEGATIVE    POSITIVE 
                               (1)        (2)       (3)       (4)       (5)       (6)         (7)         (8)   
----------------------------------------------------------------------------------------------------------------
adjusted_modelDIFF IN DIFF  0.00000***  0.465**   0.00000    0.450    0.00000    0.457      0.00000      0.463  
                            (0.00000)   (0.657)  (0.00000)  (0.656)  (0.00000)  (0.655)     (0.000)     (0.657) 
                                                                                                                
adjusted_modelFIXED EFFECT    2.271      0.804   2.203***  0.826***  2.245***  0.830***    2.277***    0.822*** 
                             (0.457)    (0.278)   (0.457)   (0.278)   (0.455)   (0.276)     (0.458)     (0.279) 
                                                                                               

test with `id` 

In [43]:
#
t_1 <- multinom(sign_of_effect ~ adjusted_model+ social  +#adjusted_dependent+
           sjr + id, data = df_final, trace = FALSE)    
t_1.rrr <- exp(coef(t_1))
#
t_2 <- multinom(sign_of_effect ~ adjusted_model+ environmnental  +#adjusted_dependent+
           sjr+ id, data = df_final, trace = FALSE)    
t_2.rrr <- exp(coef(t_2))
#
t_3 <- multinom(sign_of_effect ~ adjusted_model+ governance  +#adjusted_dependent+
           sjr+ id, data = df_final, trace = FALSE)    
t_3.rrr <- exp(coef(t_3))
#
t_4 <- multinom(sign_of_effect ~ adjusted_model+ social +environmnental+ governance  +#adjusted_dependent+
           sjr+ id, data = df_final, trace = FALSE)    
t_4.rrr <- exp(coef(t_4))

dep <- "Dependent variable: Sign insignificant"

list_final <- list(t_1, t_2, t_3, t_4)
list_final.rrr <-list(t_1.rrr, t_2.rrr, t_3.rrr, t_4.rrr)
stargazer(list_final,
          type = "text", 
          coef=list_final.rrr,
          omit = "id",
          style = "qje")


                             NEGATIVE  POSITIVE   NEGATIVE  POSITIVE   NEGATIVE       POSITIVE             NEGATIVE            POSITIVE    
                               (1)        (2)       (3)        (4)       (5)            (6)                  (7)                 (8)       
-------------------------------------------------------------------------------------------------------------------------------------------
adjusted_modelDIFF IN DIFF  0.00000*** 0.462***  0.00000***   0.473   0.00000***       0.465              0.00000***            0.469      
                             (0.000)    (0.412)   (0.000)    (0.412)   (0.000)        (0.413)              (0.000)             (0.413)     
                                                                                                                                           
adjusted_modelFIXED EFFECT    2.700      0.328    2.775**     0.329     2.629*         0.350               2.764**              0.328      
                   

In [35]:
#
t_1 <- multinom(sign_of_effect ~ adjusted_model+ social  +adjusted_dependent+
           sjr
                , data = df_final, trace = FALSE)    
t_1.rrr <- exp(coef(t_1))
#
t_2 <- multinom(sign_of_effect ~ adjusted_model+ environmnental + adjusted_dependent+
                publication_year + 
           sjr
                , data = df_final, trace = FALSE)    
t_2.rrr <- exp(coef(t_2))
#
t_3 <- multinom(sign_of_effect ~ adjusted_model+ environmnental + adjusted_dependent+
                publication_year + first_date_of_observations + last_date_of_observations +
           sjr
                , data = df_final, trace = FALSE)    
t_3.rrr <- exp(coef(t_3))
#
t_4 <- multinom(sign_of_effect ~ adjusted_model+ environmnental + adjusted_dependent+
                publication_year + windows +
           sjr
                , data = df_final, trace = FALSE)    
t_4.rrr <- exp(coef(t_4))

dep <- "Dependent variable: Sign insignificant"

list_final <- list(t_1, t_2)
list_final.rrr <-list(t_1.rrr, t_2.rrr)
stargazer(list_final,
          type = "text", 
          coef=list_final.rrr,
          omit = "id",
          style = "qje")


                                   NEGATIVE  POSITIVE   NEGATIVE  POSITIVE 
                                     (1)        (2)       (3)        (4)   
---------------------------------------------------------------------------
adjusted_modelDIFF IN DIFF        0.00000***  0.497**    0.000    0.739*** 
                                  (0.00000)   (0.679)   (0.000)    (0.029) 
                                                                           
adjusted_modelFIXED EFFECT          2.300      0.843    2.304***  0.768*** 
                                   (0.464)    (0.287)   (0.174)    (0.180) 
                                                                           
adjusted_modelINSTRUMENT            0.403      0.670    0.399***  0.660*** 
                                   (1.161)    (0.591)   (0.012)    (0.042) 
                                                                           
adjusted_modelLAG DEPENDENT        0.932***  0.051***   0.910***   0.044*  
           

In [37]:
(
    df
    .groupby(['sign_of_effect'])['adjusted_dependent']
    .value_counts()
    .rename('count')
    #.reset_index()
    #.set_index(['adjusted_dependent', 'sign_of_effect', 'adjusted_model'])
    .unstack(-2)
    #.style
    #.format("{0:,.0f}")
)

sign_of_effect,INSIGNIFICANT,NEGATIVE,POSITIVE
adjusted_dependent,,,
ABNORMAL_RETURN,5.0,NaN,13.0
CFP,11.0,11.0,11.0
OTHER,60.0,25.0,53.0
ROA,91.0,28.0,74.0
ROE,43.0,9.0,26.0
TOBIN_Q,96.0,32.0,62.0


In [38]:
(
    df
    .groupby(['adjusted_model','sign_of_effect'])['adjusted_dependent']
    .value_counts()
    .rename('count')
    .reset_index()
    .set_index(['adjusted_dependent', 'sign_of_effect', 'adjusted_model'])
    .unstack(-1)
    .style
    .format("{0:,.0f}")
)

Test with Kyoto, financial crisis & region

- CASE WHEN first_date_of_observations >= 1997 THEN TRUE ELSE FALSE END AS kyoto,
- CASE WHEN first_date_of_observations >= 2009 THEN TRUE ELSE FALSE END AS financial_crisis

In [48]:
#
t_1 <- multinom(sign_of_effect ~ adjusted_model+ environmnental + adjusted_dependent+
                publication_year + kyoto + financial_crisis+
           sjr, data = df_final, trace = FALSE)    
t_1.rrr <- exp(coef(t_1))
#
t_2 <- multinom(sign_of_effect ~ adjusted_model+ environmnental + adjusted_dependent+
                publication_year + first_date_of_observations + last_date_of_observations +
                + kyoto + financial_crisis+
           sjr, data = df_final, trace = FALSE)    
t_2.rrr <- exp(coef(t_2))
#
t_3 <- multinom(sign_of_effect ~ adjusted_model+ environmnental + adjusted_dependent+
                publication_year + windows +
                + kyoto + financial_crisis+
           sjr, data = df_final, trace = FALSE)    
t_3.rrr <- exp(coef(t_3))

dep <- "Dependent variable: Sign insignificant"

list_final <- list(t_1, t_2, t_3)
list_final.rrr <-list(t_1.rrr, t_2.rrr,  t_3.rrr)
stargazer(list_final,
          type = "text", 
          coef=list_final.rrr,
          omit = "id",
          style = "qje")


                                       NEGATIVE      POSITIVE  NEGATIVE  POSITIVE  NEGATIVE  POSITIVE 
                                         (1)            (2)       (3)       (4)       (5)       (6)   
------------------------------------------------------------------------------------------------------
adjusted_modelDIFF IN DIFF              0.000          0.826     0.000   0.635***    0.000   0.718*** 
                                                      (0.033)   (0.000)   (0.020)   (0.000)   (0.040) 
                                                                                                      
adjusted_modelFIXED EFFECT              2.465          0.804   2.835***  0.695***  2.765***  0.804*** 
                                       (0.190)        (0.195)   (0.071)   (0.085)   (0.192)   (0.201) 
                                                                                                      
adjusted_modelINSTRUMENT               0.461***      0.721***  0.493*** 

In [42]:
(
    df
    .groupby(['adjusted_model','sign_of_effect'])['financial_crisis']
    .value_counts()
    .rename('count')
    .reset_index()
    .set_index(['financial_crisis', 'sign_of_effect', 'adjusted_model'])
    .unstack(-1)
    .style
    .format("{0:,.0f}")
)

In [51]:
t_4 <- multinom(sign_of_effect ~  environmnental + adjusted_dependent+
                publication_year + windows +
                + kyoto + financial_crisis+ regions+
           sjr, data = df_final, trace = FALSE)    
t_4.rrr <- exp(coef(t_4))
stargazer(list(t_4),
          type = "text", 
          coef=list(t_4.rrr),
          omit = "id",
          style = "qje")


                                    NEGATIVE                  POSITIVE               
                                       (1)                       (2)                 
-------------------------------------------------------------------------------------
environmnentalYES                     0.960                     1.223                
                                     (0.266)                   (0.239)               
                                                                                     
adjusted_dependentABNORMAL_RETURN     0.000                   3.159***               
                                                               (0.018)               
                                                                                     
adjusted_dependentCFP               1.493***                  1.131***               
                                     (0.060)                   (0.065)               
                                                     

In [52]:
(
    df
    .groupby(['adjusted_model','sign_of_effect'])['regions']
    .value_counts()
    .rename('count')
    .reset_index()
    .set_index(['regions', 'sign_of_effect', 'adjusted_model'])
    .unstack(-1)
    .style
    .format("{0:,.0f}")
)

# Generate reports

In [ ]:
import os, time, shutil, urllib, ipykernel, json
from pathlib import Path
from notebook import notebookapp
import sys
path = os.getcwd()
parent_path = str(Path(path).parent.parent.parent)
sys.path.append(os.path.join(parent_path, 'utils'))
import make_toc
import create_report

In [ ]:
name_json = 'parameters_ETL_esg_metadata.json'
path_json = os.path.join(str(Path(path).parent.parent), 'utils',name_json)

In [ ]:
create_report.create_report(extension = "html", keep_code = True, notebookname = "00_sign_of_effect_classification.ipynb")

In [ ]:
### Update TOC in Github
for p in [parent_path,
          str(Path(path).parent),
          #os.path.join(str(Path(path).parent), "00_download_data_from"),
          #os.path.join(str(Path(path).parent.parent), "02_data_analysis"),
          #os.path.join(str(Path(path).parent.parent), "02_data_analysis", "00_statistical_exploration"),
          #os.path.join(str(Path(path).parent.parent), "02_data_analysis", "01_model_estimation"),
         ]:
    try:
        os.remove(os.path.join(p, 'README.md'))
    except:
        pass
    path_parameter = os.path.join(parent_path,'utils', name_json)
    md_lines =  make_toc.create_index(cwd = p, path_parameter = path_parameter)
    md_out_fn = os.path.join(p,'README.md')
    
    if p == parent_path:
    
        make_toc.replace_index(md_out_fn, md_lines, Header = os.path.basename(p).replace('_', ' '), add_description = True, path_parameter = path_parameter)
    else:
        make_toc.replace_index(md_out_fn, md_lines, Header = os.path.basename(p).replace('_', ' '), add_description = False)